# Measuring historic & projected temperatures in Pennsylvania with Google Earth Engine

This notebook builds on a tutorial from Michael Krisch and Juan Saldarriaga at the Brown Institute as well as existing work done by [Justin Braaten](https://jdbcode.github.io/) from the Google Earth Engine team. 


Datasets:

- Historical climate ([PRISM](https://developers.google.com/earth-engine/datasets/catalog/OREGONSTATE_PRISM_AN81m))
- Projected climate ([NEX-DCP30](https://developers.google.com/earth-engine/datasets/catalog/NASA_NEX-DCP30))


In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=jJmpMId3emv7EMMG1YjVeXUkjDVrvSxfXfXzqy8_WTk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiukoHsjSzZkJxcoK94ZsJ_TJjbrmPyGQ-SR8jmBTtOi-95lt5icgU

Successfully saved authorization token.


In [2]:
import pandas as pd
import altair as alt
import numpy as np

# Region Reduction & Formatting

In [3]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

In [4]:
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

# Import & prepare collection for DataFrame

Filtered the collection by date and scenario, and calculated the 'mean' temperature from median min and max among 33 models. 

Created a region reduction function. Applied the function to all images in the time series, and filtered out features with null computed values. Converted  to a Pandas DataFrame.




In [5]:
dcp_col = (ee.ImageCollection('NASA/NEX-DCP30_ENSEMBLE_STATS')
           .select(['tasmax_median', 'tasmin_median', 'pr_median'])
           .filter(
               ee.Filter.And(ee.Filter.eq('scenario', 'rcp85'),
                             ee.Filter.date('2020-01-01', '2070-01-01'))))

def calc_mean_temp(img):
  return (img.select('tasmax_median')
          .add(img.select('tasmin_median'))
          .divide(ee.Image.constant(2.0))
          .addBands(img.select('pr_median'))
          .rename(['Temp-mean', 'Precip-rate'])
          .copyProperties(img, img.propertyNames()))

dcp_col = dcp_col.map(calc_mean_temp)

In [6]:
aoi = ee.FeatureCollection('TIGER/2018/States').filter( ee.Filter.eq('STATEFP', '42')).geometry()

In [7]:
reduce_dcp30 = create_reduce_region_function(
    geometry=aoi, reducer=ee.Reducer.first(), scale=5000, crs='EPSG:3310')

dcp_stat_fc = ee.FeatureCollection(dcp_col.map(reduce_dcp30)).filter(
    ee.Filter.notNull(dcp_col.first().bandNames()))

In [8]:
dcp_dict = fc_to_dict(dcp_stat_fc).getInfo()
dcp_df = pd.DataFrame(dcp_dict)
display(dcp_df)
print(dcp_df.dtypes)

,Precip-rate,Temp-mean,millis,system:index
0,0.000030,273.485947,1577836800000,rcp85_202001
1,0.000031,275.641541,1580515200000,rcp85_202002
2,0.000045,278.604111,1583020800000,rcp85_202003
3,0.000037,284.687485,1585699200000,rcp85_202004
4,0.000039,289.711227,1588291200000,rcp85_202005
...,...,...,...,...
595,0.000040,300.051010,3142540800000,rcp85_206908
596,0.000028,296.175613,3145219200000,rcp85_206909
597,0.000029,289.236786,3147811200000,rcp85_206910
598,0.000036,283.033905,3150489600000,rcp85_206911


Precip-rate     float64
Temp-mean       float64
millis            int64
system:index     object
dtype: object


In [9]:
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

In [10]:
dcp_df = add_date_info(dcp_df)
dcp_df.head(5)

,Precip-rate,Temp-mean,millis,system:index,Timestamp,Year,Month,Day,DOY
0,0.000030,273.485947,1577836800000,rcp85_202001,2020-01-01,2020,1,1,1
1,0.000031,275.641541,1580515200000,rcp85_202002,2020-02-01,2020,2,1,32
2,0.000045,278.604111,1583020800000,rcp85_202003,2020-03-01,2020,3,1,61
3,0.000037,284.687485,1585699200000,rcp85_202004,2020-04-01,2020,4,1,92
4,0.000039,289.711227,1588291200000,rcp85_202005,2020-05-01,2020,5,1,122


In [11]:
dcp_df['Precip-mm'] = dcp_df['Precip-rate'] * 86400 * 30
dcp_df['Temp-mean'] = dcp_df['Temp-mean'] - 273.15
dcp_df['Model'] = 'Projected temperature'
dcp_df = dcp_df.drop('Precip-rate', 1)
dcp_df.head(5)

,Temp-mean,millis,system:index,Timestamp,Year,Month,Day,DOY,Precip-mm,Model
0,0.335947,1577836800000,rcp85_202001,2020-01-01,2020,1,1,1,78.389837,Projected temperature
1,2.491541,1580515200000,rcp85_202002,2020-02-01,2020,2,1,32,79.207561,Projected temperature
2,5.454111,1583020800000,rcp85_202003,2020-03-01,2020,3,1,61,116.136855,Projected temperature
3,11.537485,1585699200000,rcp85_202004,2020-04-01,2020,4,1,92,96.558770,Projected temperature
4,16.561227,1588291200000,rcp85_202005,2020-05-01,2020,5,1,122,101.487229,Projected temperature


# Reduce collection & Prepare DataFrame for historical climate data

Reduced the collection images by region and filtered for null values. Converted the feature collection to a dictionary and then to a DataFrame. Added date columns and model name. Renamed columns to be consistent with the NEX-DCP30 DataFrame.

In [12]:
prism_col = (ee.ImageCollection('OREGONSTATE/PRISM/AN81m')
             .select(['ppt', 'tmean'])
             .filter(ee.Filter.date('1979-01-01', '2020-12-31')))

reduce_prism = create_reduce_region_function(
    geometry=aoi, reducer=ee.Reducer.first(), scale=5000, crs='EPSG:3310')

prism_stat_fc = (ee.FeatureCollection(prism_col.map(reduce_prism))
                 .filter(ee.Filter.notNull(prism_col.first().bandNames())))

prism_dict = fc_to_dict(prism_stat_fc).getInfo()
prism_df = pd.DataFrame(prism_dict)

display(prism_df)
print(prism_df.dtypes)

,millis,ppt,system:index,tmean
0,283996800000,161.169998,197901,-4.985000
1,286675200000,95.260002,197902,-7.195000
2,289094400000,66.940002,197903,5.270000
3,291772800000,103.180000,197904,9.370000
4,294364800000,152.809998,197905,14.220000
...,...,...,...,...
499,1596240000000,82.973000,202008,22.854399
500,1598918400000,68.010002,202009,17.916000
501,1601510400000,73.773003,202010,12.005400
502,1604188800000,59.880001,202011,7.506400


millis            int64
ppt             float64
system:index     object
tmean           float64
dtype: object


In [13]:
prism_df = add_date_info(prism_df)
prism_df['Model'] = 'Historic temperature'
prism_df = prism_df.rename(columns={'ppt': 'Precip-mm', 'tmean': 'Temp-mean'})
prism_df.tail(5)

,millis,Precip-mm,system:index,Temp-mean,Timestamp,Year,Month,Day,DOY,Model
499,1596240000000,82.973000,202008,22.854399,2020-08-01,2020,8,1,214,Historic temperature
500,1598918400000,68.010002,202009,17.916000,2020-09-01,2020,9,1,245,Historic temperature
501,1601510400000,73.773003,202010,12.005400,2020-10-01,2020,10,1,275,Historic temperature
502,1604188800000,59.880001,202011,7.506400,2020-11-01,2020,11,1,306,Historic temperature
503,1606780800000,113.097000,202012,0.439400,2020-12-01,2020,12,1,336,Historic temperature


# Combine DataFrames

Concatenated these DataFrames into a single DataFrame for plotting together in the same chart, and converted temperature to Farenheight.

In [14]:
climate_df = pd.concat([prism_df, dcp_df], sort=True)
climate_df

,DOY,Day,Model,Month,Precip-mm,Temp-mean,Timestamp,Year,millis,system:index
0,1,1,Historic temperature,1,161.169998,-4.985000,1979-01-01,1979,283996800000,197901
1,32,1,Historic temperature,2,95.260002,-7.195000,1979-02-01,1979,286675200000,197902
2,60,1,Historic temperature,3,66.940002,5.270000,1979-03-01,1979,289094400000,197903
3,91,1,Historic temperature,4,103.180000,9.370000,1979-04-01,1979,291772800000,197904
4,121,1,Historic temperature,5,152.809998,14.220000,1979-05-01,1979,294364800000,197905
...,...,...,...,...,...,...,...,...,...,...
595,213,1,Projected temperature,8,103.680356,26.901010,2069-08-01,2069,3142540800000,rcp85_206908
596,244,1,Projected temperature,9,73.320485,23.025613,2069-09-01,2069,3145219200000,rcp85_206909
597,274,1,Projected temperature,10,74.771066,16.086786,2069-10-01,2069,3147811200000,rcp85_206910
598,305,1,Projected temperature,11,93.995153,9.883905,2069-11-01,2069,3150489600000,rcp85_206911


In [15]:
climate_df['Temp-mean'] = (climate_df['Temp-mean']*(9/5)) + 32

In [16]:
climate_df = climate_df.rename(columns = {'Model':'Climate model'})

In [17]:
climate_df.to_csv('hist_proj_temp.csv')

# Charting Temperature

In [18]:
line = alt.Chart(climate_df).mark_line().encode(
    x='Year:O',
    y='median(Temp-mean):Q',
    color=alt.Color('Climate model', scale=alt.Scale(domain=['Historic temperature', 'Projected temperature'],range=['grey', '#D0372D'])))

band = alt.Chart(climate_df).mark_errorband(extent='iqr').encode(
    x='Year:O',
    y=alt.Y('Temp-mean:Q', title='Temperature (°F)'), 
    color=alt.Color('Climate model', scale=alt.Scale(domain=['Historic temperature', 'Projected temperature'],range=['grey', '#D0372D'])))

(band + line).properties(width=600, height=300)

alt.LayerChart(...)